# Corner Bracket

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from ansys.mapdl.core import launch_mapdl

In [2]:
mapdl = launch_mapdl()

In [3]:
print(mapdl)

Product:             Ansys Mechanical Enterprise Academic Student
MAPDL Version:       24.2
ansys.mapdl Version: 0.68.6



In [4]:
mapdl.prep7()

*** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2024 R2          24.2     ***
 Ansys Mechanical Enterprise Academic Student      
 01055371  VERSION=WINDOWS x64   16:42:14  NOV 18, 2024 CP=      0.156

                                                                               



          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****

In [5]:
box1 = [0, 6, -1, 1]
box2 = [4, 6, -1, -3]
mapdl.rectng(*box1)  # prints the id of the created area
mapdl.rectng(*box2)  # prints the id of the created area

2

In [6]:
mapdl.aplot(cpos="xy", show_lines=True)

Widget(value='<iframe src="http://localhost:52215/index.html?ui=P_0x1e72f6856c0_0&reconnect=auto" class="pyvis…

In [7]:
radius = 1
circle1_X = box1[0]
circle1_Y = (box1[2] + box1[3]) / 2
mapdl.cyl4(circle1_X, circle1_Y, radius)

mapdl.aplot(vtk=True, cpos="xy", show_lines=True)

Widget(value='<iframe src="http://localhost:52215/index.html?ui=P_0x1e72f687280_1&reconnect=auto" class="pyvis…

In [8]:
circle2_X = (box2[0] + box2[1]) / 2
circle2_Y = box2[3]
mapdl.cyl4(circle2_X, circle2_Y, radius)
mapdl.aadd("all")

5

In [9]:
line1 = mapdl.lsel("S", "LOC", "Y", box1[2])
l1 = mapdl.get("line1", "LINE", 0, "NUM", "MAX")

In [10]:
line2 = mapdl.lsel("S", "LOC", "X", box2[0])
l2 = mapdl.get("line2", "LINE", 0, "NUM", "MAX")

In [11]:
fillet_radius = 0.4
mapdl.allsel()
line3 = mapdl.lfillt("line1", l2, fillet_radius)

In [12]:
mapdl.allsel()
mapdl.lplot(vtk=True, cpos="xy")

Widget(value='<iframe src="http://localhost:52215/index.html?ui=P_0x1e739db4a90_2&reconnect=auto" class="pyvis…

In [13]:
mapdl.allsel()

# Select lines for the area
mapdl.lsel("S", "LENGTH", "", fillet_radius)

array([4, 5])

In [14]:
mapdl.lsel("A", "LINE", "", line3)

# plotting ares
mapdl.lplot(vtk=True, cpos="xy", show_line_numbering=True)

Widget(value='<iframe src="http://localhost:52215/index.html?ui=P_0x1e739db6470_3&reconnect=auto" class="pyvis…

In [15]:
mapdl.al("ALL")  # Prints the ID of the newly created area
mapdl.lplot(vtk=True, cpos="xy", show_line_numbering=True)

Widget(value='<iframe src="http://localhost:52215/index.html?ui=P_0x1e73f08d7e0_4&reconnect=auto" class="pyvis…

In [16]:
# Add the area to the main area
mapdl.aadd("all")
mapdl.aplot(vtk=True, cpos="xy", show_lines=True)

Widget(value='<iframe src="http://localhost:52215/index.html?ui=P_0x1e7423387f0_5&reconnect=auto" class="pyvis…

In [17]:
# Create the first pinhole
pinhole_radius = 0.4
pinhole1_X = box1[0]
pinhole1_Y = (box1[2] + box1[3]) / 2

pinhole1 = mapdl.cyl4(pinhole1_X, pinhole1_Y, pinhole_radius)

pinhole2_X = (box2[0] + box2[1]) / 2
pinhole2_Y = box2[3]

pinhole2 = mapdl.cyl4(pinhole2_X, pinhole2_Y, pinhole_radius)
pinhole2_lines = mapdl.asll("S", 0)

mapdl.asba("all", pinhole1)
bracket = mapdl.asba("all", pinhole2)
mapdl.aplot(vtk=True, show_lines=True, cpos="xy")

Widget(value='<iframe src="http://localhost:52215/index.html?ui=P_0x1e74233b5e0_6&reconnect=auto" class="pyvis…

In [18]:
ex = 30e6  # Young's Modulus
prxy = 0.27  # Poisson's ratio

mapdl.mp("EX", 1, ex)
mapdl.mp("PRXY", 1, prxy)

MATERIAL          1     PRXY =  0.2700000

In [ ]:
# define a ``PLANE183`` element type with thickness
mapdl.et(1, "PLANE183", kop3=3)
# Set element thickness
thick = 0.5
mapdl.r(1, thick)  # thickness of 0.5 length units)

REAL CONSTANT SET          1  ITEMS   1 TO   6
   0.50000       0.0000       0.0000       0.0000       0.0000       0.0000

In [20]:
element_size = 0.5
mapdl.esize(element_size)
mapdl.amesh(bracket)
mapdl.eplot(
    vtk=True,
    cpos="xy",
    show_edges=True,
    show_axes=False,
    line_width=2,
    background="w",
)

Widget(value='<iframe src="http://localhost:52215/index.html?ui=P_0x1e745419e40_7&reconnect=auto" class="pyvis…

In [21]:
mapdl.allsel()
mapdl.solution() # call solution class
mapdl.antype("STATIC")


PERFORM A STATIC ANALYSIS
  THIS WILL BE A NEW ANALYSIS

In [22]:
mapdl.lplot(vtk=True, cpos="xy", show_line_numbering=True)

Widget(value='<iframe src="http://localhost:52215/index.html?ui=P_0x1e75e78bbb0_8&reconnect=auto" class="pyvis…

In [23]:
bc1 = mapdl.lsel(
    "S", "LOC", "X", pinhole1_X - pinhole_radius, pinhole1_X + pinhole_radius
)
print(f"Number of lines selected : {len(bc1)}")

Number of lines selected : 4


In [25]:
fixNodes = mapdl.nsll(type_="S")

In [26]:
fixNodes

SELECT      ONLY NODES INTERIOR TO SELECTED LINE SET.

         12  NODES (OF        335  DEFINED) SELECTED FROM 
        4 SELECTED LINES BY NSLL COMMAND.

In [27]:
# Set up boundary conditions
mapdl.d("ALL", "ALL", 0)  # The 0 is not required since default is zero

# Selecting everything again
mapdl.allsel()

SELECT ALL ENTITIES OF TYPE= ALL  AND BELOW

In [28]:
p1 = 50
p2 = 500

In [29]:
mapdl.lsel("S", "LOC", "Y", pinhole2_Y - pinhole_radius, pinhole2_Y)


array([11, 12])

In [30]:
mapdl.lsel("R", "LOC", "X", 0, pinhole2_X)

mapdl.lplot(vtk=True, cpos="xy")

Widget(value='<iframe src="http://localhost:52215/index.html?ui=P_0x1e75e78a1a0_9&reconnect=auto" class="pyvis…

In [31]:
# Here you load the left side of the lower half of second pin hole.
mapdl.sf("ALL", "PRES", p1, p2)
mapdl.allsel()
mapdl.lsel("S", "LOC", "Y", pinhole2_Y - pinhole_radius, pinhole2_Y)
mapdl.lsel("R", "LOC", "X", pinhole2_X, pinhole2_X + pinhole_radius)

mapdl.lplot(
    vtk=True,
    cpos="xy",
    show_line_numbering=True,
)

mapdl.sf("ALL", "PRES", p2, p1)
mapdl.allsel()

Widget(value='<iframe src="http://localhost:52215/index.html?ui=P_0x1e702109990_10&reconnect=auto" class="pyvi…

SELECT ALL ENTITIES OF TYPE= ALL  AND BELOW

In [32]:
# Solve the model
output = mapdl.solve()
print(output)

*****  MAPDL SOLVE    COMMAND  *****

 *** NOTE ***                            CP =       6.656   TIME= 16:58:52
 There is no title defined for this analysis.                            

 *** SELECTION OF ELEMENT TECHNOLOGIES FOR APPLICABLE ELEMENTS ***
                ---GIVE SUGGESTIONS ONLY---

 ELEMENT TYPE         1 IS PLANE183 WITH PLANE STRESS OPTION. NO SUGGESTION IS
 AVAILABLE.



 *** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2024 R2          24.2     ***
 Ansys Mechanical Enterprise Academic Student      
 01055371  VERSION=WINDOWS x64   16:58:52  NOV 18, 2024 CP=      6.656

                                                                               



                       S O L U T I O N   O P T I O N S

   PROBLEM DIMENSIONALITY. . . . . . . . . . . . .2-D                  
   DEGREES OF FREEDOM. . . . . . UX   UY  
   ANALYSIS TYPE . . . . . . . . . . . . . . . . .STATIC (STEADY-STATE)
   GLOBALLY ASSEMBLED MATRIX . . . . . . . . . . .SYMMETRIC  

 *** NOTE ***

In [33]:
mapdl.post1()

FINISH SOLUTION PROCESSING


 ***** ROUTINE COMPLETED *****  CP =         6.719



 *** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2024 R2          24.2     ***
 Ansys Mechanical Enterprise Academic Student      
 01055371  VERSION=WINDOWS x64   16:59:42  NOV 18, 2024 CP=      6.719

                                                                               



          ***** MAPDL RESULTS INTERPRETATION (POST1) *****

 *** NOTE ***                            CP =       6.719   TIME= 16:59:42
 Reading results into the database (SET command) will update the current 
 displacement and force boundary conditions in the database with the     
 values from the results file for that load set.  Note that any          
 subsequent solutions will use these values unless action is taken to    
 either SAVE the current values or not overwrite them (/EXIT,NOSAVE).

In [34]:
# Plot displacement
result = mapdl.result
result_set = 0  # Plotting the first results
disp_fact = 1e10
result.plot_nodal_displacement(
    result_set,
    cpos="xy",
    displacement_factor=5,
    show_displacement=True,
    show_edges=True,
)

Widget(value='<iframe src="http://localhost:52215/index.html?ui=P_0x1e75e78baf0_11&reconnect=auto" class="pyvi…

In [35]:
result.plot_principal_nodal_stress(
    0,
    "SEQV",
    cpos="xy",
    background="w",
    text_color="k",
    add_text=True,
    show_edges=True,
)

Widget(value='<iframe src="http://localhost:52215/index.html?ui=P_0x1e7021dc340_12&reconnect=auto" class="pyvi…

In [36]:
nnum, stress = result.principal_nodal_stress(0)

In [47]:
reactForces = mapdl.prrsol(lab="FY").to_dataframe(columns=["NODE", "FY"])

In [48]:
reactForces

,NODE,FY
0,103.0,21.7590
1,104.0,9.8719
2,105.0,12.5870
3,107.0,1.5168
4,108.0,12.2310
5,109.0,29.4880
6,111.0,-29.8790
7,112.0,-10.5920
8,113.0,-2.5094
9,114.0,-12.4730


In [1]:
mapdl.exit()


NameError: name 'mapdl' is not defined